In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ../..

/media/samuele/Disco/PycharmProjectsUbuntu/MARL-CER


In [2]:
import jax
import jax.numpy as jnp
from flax import nnx
from functools import partial
from jax_tqdm import scan_tqdm

from algorithms.utils import save_state_multiagent

import numpy as np

# jax.config.update("jax_debug_nans", True)

2025-03-04 09:49:53.726330: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741078193.739217   22538 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741078193.743182   22538 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from ernestogym.envs_jax.multi_agent.env import RECEnv

In [4]:
def my_env_creator(params, battery_type):
    env = RECEnv(params, battery_type)
    return env

In [5]:
from ernestogym.envs_jax.multi_agent.utils import parameter_generator

In [6]:
pack_options = "ernestogym/ernesto_jax/data/battery_new/pack.yaml"
ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_pack.yaml"
# ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_fading_pack.yaml"
r2c = "ernestogym/ernesto_jax/data/battery_new/models/thermal/r2c_thermal_pack.yaml"
# bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"
bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"
world = "ernestogym/envs_jax/multi_agent/world_deg_train.yaml"
# world = "ernestogym/envs_jax/single_agent/world_fading.yaml"

# battery_type = 'fading'
# battery_type = 'degrading'
battery_type = 'degrading_dropflow'

params = parameter_generator(
    input_var='current',
    battery_options=pack_options,
    electrical_model=ecm,
    thermal_model=r2c,
    aging_model=bolun,
    world_options=world,
    use_reward_normalization=True

)

params

{'num_battery_agents': 5,
 'num_passive_houses': 0,
 'batteries': [{'sign_convention': 'passive',
   'params': {'nominal_voltage': 350.4,
    'nominal_capacity': 60.0,
    'nominal_dod': 0.8,
    'nominal_lifetime': 3000,
    'v_max': 398.4,
    'v_min': 288.0,
    'temp_ambient': 298.15,
    'nominal_cost': 3000.0},
   'bounds': {'voltage': {'low': 288.0, 'high': 398.4},
    'current': {'low': -180.0, 'high': 60.0},
    'power': {'low': -71712.0, 'high': 23904.0},
    'temperature': {'low': 273.15, 'high': 323.15},
    'temp_ambient': {'low': 273.15, 'high': 323.15},
    'soc': {'low': 0.2, 'high': 1.0},
    'soh': {'low': 0.6, 'high': 1.0}},
   'init': {'voltage': 310.8,
    'current': 0.0,
    'power': 0.0,
    'temperature': 293.15,
    'temp_ambient': 293.15,
    'soc': 0.2,
    'soh': 1.0}},
  {'sign_convention': 'passive',
   'params': {'nominal_voltage': 350.4,
    'nominal_capacity': 60.0,
    'nominal_dod': 0.8,
    'nominal_lifetime': 3000,
    'v_max': 398.4,
    'v_min': 2

In [7]:
pack_options = "ernestogym/ernesto_jax/data/battery_new/pack.yaml"
ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_pack.yaml"
# ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_fading_pack.yaml"
r2c = "ernestogym/ernesto_jax/data/battery_new/models/thermal/r2c_thermal_pack.yaml"
# bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"
bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"
world = "ernestogym/envs_jax/multi_agent/world_deg_test.yaml"
# world = "ernestogym/envs_jax/single_agent/world_fading.yaml"

params_testing = parameter_generator(
    input_var='current',
    battery_options=pack_options,
    electrical_model=ecm,
    thermal_model=r2c,
    aging_model=bolun,
    world_options=world,
    use_reward_normalization=True

)

params_testing['step']

3600

In [8]:
from algorithms.multi_agent_ppo import make_train, train        #, train_for, train_for_flax
from flax.core.frozen_dict import freeze, unfreeze

In [9]:
num_envs = 4

config = {
    "LR_BATTERIES": 5e-3,
    "LR_REC": 5e-3,
    "LR_BATTERIES_MIN": 5e-4,
    "LR_REC_MIN": 5e-4,
    "NUM_ENVS": num_envs,
    "NUM_STEPS": 8192, #2048 #10000,
    # "TOTAL_TIMESTEPS": 100000 * num_envs,
    "TOTAL_TIMESTEPS": 876000 * num_envs,
    "UPDATE_EPOCHS": 10,
    "NUM_MINIBATCHES": 32, #64,
    "NORMALIZE_ENV": False,
    "GAMMA": 0.99,
    "GAE_LAMBDA": 0.95,
    "CLIP_EPS": 0.2,
    "ENT_COEF": 0.01,
    "VF_COEF": 0.5,
    "MAX_GRAD_NORM": 0.5,
    "NET_ARCH": (64, 32, 16), #(16, 16, 16),
    "LSTM_NET_ARCH": (24, 12),
    "ACTIVATION": 'tanh',
    "LSTM_ACTIVATION": 'tanh',
    "LOGISTIC_FUNCTION_TO_ACTOR": False,
    "ENV_NAME": None,
    "LR_SCHEDULE": 'cosine',
    'DEBUG': False,
    "NORMALIZE_ADVANTAGES": True,
    "ENV_TYPE": 'normal',
    'NETWORK_TYPE_BATTERIES': 'recurrent_actor_critic',
    'NETWORK_TYPE_REC': 'recurrent_actor_critic',
    'USE_WEIGHT_DECAY': True
}

env_type = config['ENV_TYPE']

rng = jax.random.PRNGKey(42)

env = my_env_creator(params, battery_type)
env, train_state = make_train(config, env)

env_testing = my_env_creator(params_testing, battery_type)

val_rng = jax.random.PRNGKey(51)
val_num_iters = 8670*5

# out = train_jit(rng)

config = freeze(config)

In [10]:
env.battery_agents

['battery_agent_0',
 'battery_agent_1',
 'battery_agent_2',
 'battery_agent_3',
 'battery_agent_4']

In [11]:
networks_batteries, optimizer_batteries, network_rec, optimizer_rec = nnx.merge(train_state.graph_def, train_state.state)

In [12]:
networks_batteries

StackedRecurrentActorCritic(
  act_layers=[Linear(
    bias=Param(
      value=Array(shape=(5, 64), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x78995c4e31a0>,
    dot_general=<function dot_general at 0x789a1da63c40>,
    dtype=None,
    in_features=16,
    kernel=Param(
      value=Array(shape=(5, 16, 64), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x78995c4e2c00>,
    out_features=64,
    param_dtype=<class 'jax.numpy.float32'>,
    precision=None,
    use_bias=True
  ), <PjitFunction of <function tanh at 0x789a1d736840>>, Linear(
    bias=Param(
      value=Array(shape=(5, 32), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x78995c583c40>,
    dot_general=<function dot_general at 0x789a1da63c40>,
    dtype=None,
    in_features=64,
    kernel=Param(
      value=Array(shape=(5, 64, 32), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x78995c5834c0>,
    out_features=32

In [13]:
network_rec

RECRecurrentActorCritic(
  act_layers=[Linear(
    bias=Param(
      value=Array(shape=(64,), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x78995c109580>,
    dot_general=<function dot_general at 0x789a1da63c40>,
    dtype=None,
    in_features=16,
    kernel=Param(
      value=Array(shape=(16, 64), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x78995c108860>,
    out_features=64,
    param_dtype=<class 'jax.numpy.float32'>,
    precision=None,
    use_bias=True
  ), <PjitFunction of <function tanh at 0x789a1d736840>>, Linear(
    bias=Param(
      value=Array(shape=(32,), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x78995c10a2a0>,
    dot_general=<function dot_general at 0x789a1da63c40>,
    dtype=None,
    in_features=64,
    kernel=Param(
      value=Array(shape=(64, 32), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x78995c10a840>,
    out_features=32,
    param_dt

In [ ]:
import time
rng = jax.random.PRNGKey(42)
t0 = time.time()

# with jax.checking_leaks():
out = train(env, config, train_state, rng, validate=True, freq_val=10, val_env=env_testing, val_rng=val_rng, val_num_iters=val_num_iters)

train_state = out['runner_state'].train_state

print(f"time: {time.time() - t0:.2f} s")

2025-03-04 09:50:50.655408: W external/xla/xla/hlo/transforms/simplifiers/hlo_rematerialization.cc:3021] Can't reduce memory use below 1.05GiB (1126623203 bytes) by rematerialization; only reduced to 2.00GiB (2148553242 bytes), down from 2.00GiB (2149188338 bytes) originally


Running for 43,350 iterations:   0%|          | 0/43350 [00:00<?, ?it/s]

bat loss [[3.4827456  1.3074373  1.178824   1.3193158  1.3112065  1.0322952
  1.1297169  0.9217439  1.6822205  1.0354416  2.0335834  1.3140769
  0.71828085 1.7674971  0.66116273 0.75411916 0.79330784 0.52376544
  0.9566963  2.0793083  0.76460505 0.71891457 2.1173096  1.4467069
  0.8614063  1.1758637  1.0535537  0.9795031  1.363428   1.4187356
  1.4294109  0.8462752 ]
 [0.6209718  1.9817075  2.7488697  0.8880024  0.61359185 1.099303
  0.87914026 0.95202076 0.77872556 1.7394369  0.9926574  1.4079654
  1.1806955  1.2766819  0.9657782  1.6407977  0.724596   0.59888744
  0.967114   1.362517   0.8213067  1.3658874  0.71435994 1.1919347
  1.1740458  1.4035811  2.0186627  0.5098172  0.9340233  2.1143503
  1.275819   1.077259  ]
 [1.4051597  1.0390122  0.96779776 1.2690661  0.93400884 1.1962731
  0.49611342 1.1429449  1.7290602  0.8956082  2.7404475  1.2824498
  1.3510652  0.5925659  1.9777584  1.1585999  0.5954516  1.6455028
  2.1091282  0.7065573  1.0648822  0.85756075 1.4062147  0.9432396
  

Running for 106 iterations:   0%|          | 0/106 [00:00<?, ?it/s]

Running for 43,350 iterations:   0%|          | 0/43350 [00:00<?, ?it/s]

curr_iter: 0
	r_tot: -14213.072265625
	r_trad: -5491.0703125
	r_deg: -5422.251953125
	r_clip: -3603.26416015625
	r_glob: 303.51361083984375
	r_rec: 2494981.75
bat loss [[1.2073264  1.2822233  0.91009605 0.5344009  1.5413247  0.8029955
  1.367116   0.7297914  0.68447596 0.66667795 1.3386959  0.84686655
  0.8424325  0.9455112  1.0374653  0.6049664  0.94617724 0.60773385
  1.2897143  0.919194   0.9063966  1.7374465  1.378995   0.9658858
  0.81989145 1.90362    0.72609454 2.0922434  0.68008286 0.7094911
  1.4938186  1.6928852 ]
 [0.8731481  0.9592862  1.2998058  0.9111189  2.090463   1.6824906
  1.3561417  0.7208464  0.829963   0.8745602  0.7857525  1.32653
  0.94290954 0.84011424 0.6617655  1.3610529  0.85732293 0.5940666
  0.822461   0.7116447  1.3738073  0.699926   0.65154    1.905426
  1.736567   0.44008294 0.6681175  0.96731997 0.6084513  1.4899057
  1.0207043  0.6110608 ]
 [0.88640755 1.3459504  2.0906327  1.670613   1.324035   1.4780848
  0.6623483  0.87660766 0.9390011  1.3609004  

In [15]:
jax.tree.map(lambda x: x.shape, out['metrics'])

{'actions': {'REC_agent': (106, 8192, 4, 5),
  'battery_agent_0': (106, 8192, 4, 1),
  'battery_agent_1': (106, 8192, 4, 1),
  'battery_agent_2': (106, 8192, 4, 1),
  'battery_agent_3': (106, 8192, 4, 1),
  'battery_agent_4': (106, 8192, 4, 1)},
 'buy_prices': (106, 8192, 4, 5),
 'demands': (106, 8192, 4, 5),
 'generations': (106, 8192, 4, 5),
 'norm_reward': {'r_clipping': (106, 8192, 4, 5),
  'r_deg': (106, 8192, 4, 5),
  'r_op': (106, 8192, 4, 5),
  'r_trad': (106, 8192, 4, 5)},
 'pure_reward': {'r_clipping': (106, 8192, 4, 5),
  'r_deg': (106, 8192, 4, 5),
  'r_op': (106, 8192, 4, 5),
  'r_trad': (106, 8192, 4, 5)},
 'r_glob': (106, 8192, 4, 5),
 'r_tot': (106, 8192, 4, 5),
 'rec_reward': (106, 8192, 4),
 'self_consumption': (106, 8192, 4),
 'sell_prices': (106, 8192, 4, 5),
 'soc': (106, 8192, 4, 5),
 'soh': (106, 8192, 4, 5),
 'tot_incentives': (106, 8192, 4),
 'weig_reward': {'r_clipping': (106, 8192, 4, 5),
  'r_deg': (106, 8192, 4, 5),
  'r_op': (106, 8192, 4, 5),
  'r_trad': 

In [16]:
jax.tree.map(lambda x: jnp.isnan(x).any(), out['metrics'])

{'actions': {'REC_agent': Array(False, dtype=bool),
  'battery_agent_0': Array(False, dtype=bool),
  'battery_agent_1': Array(False, dtype=bool),
  'battery_agent_2': Array(False, dtype=bool),
  'battery_agent_3': Array(False, dtype=bool),
  'battery_agent_4': Array(False, dtype=bool)},
 'buy_prices': Array(False, dtype=bool),
 'demands': Array(False, dtype=bool),
 'generations': Array(False, dtype=bool),
 'norm_reward': {'r_clipping': Array(False, dtype=bool),
  'r_deg': Array(False, dtype=bool),
  'r_op': Array(False, dtype=bool),
  'r_trad': Array(False, dtype=bool)},
 'pure_reward': {'r_clipping': Array(False, dtype=bool),
  'r_deg': Array(False, dtype=bool),
  'r_op': Array(False, dtype=bool),
  'r_trad': Array(False, dtype=bool)},
 'r_glob': Array(False, dtype=bool),
 'r_tot': Array(False, dtype=bool),
 'rec_reward': Array(False, dtype=bool),
 'self_consumption': Array(False, dtype=bool),
 'sell_prices': Array(False, dtype=bool),
 'soc': Array(False, dtype=bool),
 'soh': Array(Fa

In [17]:
jax.tree.map(lambda x: jnp.isnan(x).any(axis=jnp.arange(1, x.ndim)), out['metrics'])

{'actions': {'REC_agent': Array([False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False], dtype=bool),
  'battery_agent_0': Array([False, False, False, False, False, False, False, False, False,
         False, False

In [18]:
out['metrics']['actions']['REC_agent'][0]

Array([[[2.6548988e-01, 4.7862750e-02, 1.7407434e-01, 4.2843193e-01,
         8.4141091e-02],
        [7.6617157e-01, 8.8886562e-04, 3.4478164e-04, 6.7161359e-02,
         1.6543341e-01],
        [1.9753724e-01, 5.6525314e-01, 1.2317914e-02, 1.7719908e-01,
         4.7692712e-02],
        [4.2660084e-02, 1.4144063e-01, 2.1062216e-02, 7.3091352e-01,
         6.3923627e-02]],

       [[7.0307678e-01, 2.0724235e-01, 7.0552334e-02, 1.0384267e-02,
         8.7442677e-03],
        [3.6002067e-01, 2.8283605e-01, 1.5801583e-01, 1.1376758e-02,
         1.8775068e-01],
        [1.3048400e-01, 6.1342984e-01, 4.0500812e-02, 1.5336039e-02,
         2.0024934e-01],
        [5.7245022e-01, 2.5123775e-01, 9.3791515e-02, 5.9000640e-03,
         7.6620460e-02]],

       [[2.6016252e-02, 8.0101557e-02, 2.6096702e-01, 1.7518392e-01,
         4.5773125e-01],
        [2.6484281e-01, 1.1544961e-01, 5.6023026e-01, 4.8987772e-03,
         5.4578587e-02],
        [4.7067636e-01, 5.3464044e-02, 2.1882890e-01, 2.

In [19]:
out['metrics']['actions']['REC_agent'][1]

Array([[[1.42490175e-02, 7.18496680e-01, 3.06710266e-02, 8.07318762e-02,
         1.55851349e-01],
        [1.31817564e-01, 8.89364034e-02, 2.91098267e-01, 4.68488522e-02,
         4.41298842e-01],
        [9.04344581e-03, 4.71651088e-03, 6.22534990e-01, 1.81303874e-01,
         1.82401195e-01],
        [1.14682227e-01, 2.54442155e-01, 1.06059246e-01, 3.06943595e-01,
         2.17872813e-01]],

       [[1.90115362e-01, 3.79129723e-02, 6.77704513e-01, 5.22065461e-02,
         4.20606472e-02],
        [7.76720196e-02, 4.88493979e-01, 7.44709820e-02, 2.00088054e-01,
         1.59274891e-01],
        [6.83671236e-02, 4.37443495e-01, 1.19251907e-01, 3.32598239e-01,
         4.23392095e-02],
        [1.51228040e-01, 1.66567713e-01, 1.44341543e-01, 4.55059260e-01,
         8.28034356e-02]],

       [[3.32588434e-01, 3.39740604e-01, 2.47924402e-01, 1.77842882e-02,
         6.19622171e-02],
        [2.83820778e-01, 2.36591473e-01, 1.51443228e-01, 2.42826834e-01,
         8.53177011e-02],
      

In [20]:
np.isnan(out['metrics']['actions']['REC_agent'][1]).any()

False

In [21]:
jax.tree.map(lambda x: np.nonzero(np.isnan(x)), out['metrics'])

{'actions': {'REC_agent': (array([], dtype=int64),
   array([], dtype=int64),
   array([], dtype=int64),
   array([], dtype=int64)),
  'battery_agent_0': (array([], dtype=int64),
   array([], dtype=int64),
   array([], dtype=int64),
   array([], dtype=int64)),
  'battery_agent_1': (array([], dtype=int64),
   array([], dtype=int64),
   array([], dtype=int64),
   array([], dtype=int64)),
  'battery_agent_2': (array([], dtype=int64),
   array([], dtype=int64),
   array([], dtype=int64),
   array([], dtype=int64)),
  'battery_agent_3': (array([], dtype=int64),
   array([], dtype=int64),
   array([], dtype=int64),
   array([], dtype=int64)),
  'battery_agent_4': (array([], dtype=int64),
   array([], dtype=int64),
   array([], dtype=int64),
   array([], dtype=int64))},
 'buy_prices': (array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)),
 'demands': (array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=

In [22]:
out['metrics']['actions']['REC_agent']

Array([[[[2.65489876e-01, 4.78627495e-02, 1.74074337e-01,
          4.28431928e-01, 8.41410905e-02],
         [7.66171575e-01, 8.88865616e-04, 3.44781642e-04,
          6.71613589e-02, 1.65433407e-01],
         [1.97537243e-01, 5.65253139e-01, 1.23179136e-02,
          1.77199081e-01, 4.76927124e-02],
         [4.26600836e-02, 1.41440630e-01, 2.10622158e-02,
          7.30913520e-01, 6.39236271e-02]],

        [[7.03076780e-01, 2.07242355e-01, 7.05523342e-02,
          1.03842672e-02, 8.74426775e-03],
         [3.60020667e-01, 2.82836050e-01, 1.58015832e-01,
          1.13767581e-02, 1.87750682e-01],
         [1.30484000e-01, 6.13429844e-01, 4.05008122e-02,
          1.53360385e-02, 2.00249344e-01],
         [5.72450221e-01, 2.51237750e-01, 9.37915146e-02,
          5.90006402e-03, 7.66204596e-02]],

        [[2.60162521e-02, 8.01015571e-02, 2.60967016e-01,
          1.75183922e-01, 4.57731247e-01],
         [2.64842808e-01, 1.15449607e-01, 5.60230255e-01,
          4.89877723e-03, 5.4

In [23]:
out['metrics']['pure_reward']['r_clipping'][3]

Array([[[   0.      ,    0.      ,    0.      ,    0.      ,
         -198.11812 ],
        [   0.      ,    0.      ,    0.      ,    0.      ,
            0.      ],
        [   0.      ,    0.      ,    0.      ,    0.      ,
            0.      ],
        [   0.      ,    0.      ,    0.      ,    0.      ,
            0.      ]],

       [[  -7.746008,    0.      ,    0.      ,    0.      ,
            0.      ],
        [   0.      , -151.99936 ,    0.      ,    0.      ,
            0.      ],
        [   0.      ,    0.      ,    0.      ,    0.      ,
            0.      ],
        [   0.      ,    0.      ,    0.      ,    0.      ,
            0.      ]],

       [[-216.15175 ,    0.      ,    0.      ,    0.      ,
            0.      ],
        [   0.      , -161.59142 ,    0.      ,    0.      ,
            0.      ],
        [   0.      ,    0.      ,    0.      ,    0.      ,
            0.      ],
        [   0.      ,    0.      ,    0.      ,    0.      ,
           

In [24]:
network_batteries, optimizer_batteries, network_rec, optimizer_rec = nnx.merge(train_state.graph_def, train_state.state)
network_batteries

StackedActorCritic(
  act_layers=[Linear(
    bias=Param(
      value=Array(shape=(5, 64), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x74ea44379f80>,
    dot_general=<function dot_general at 0x74eaf64bfce0>,
    dtype=None,
    in_features=10,
    kernel=Param(
      value=Array(shape=(5, 10, 64), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x74ea44379620>,
    out_features=64,
    param_dtype=<class 'jax.numpy.float32'>,
    precision=None,
    use_bias=True
  ), <PjitFunction of <function tanh at 0x74eaf619e8e0>>, Linear(
    bias=Param(
      value=Array(shape=(5, 32), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x74ea44264540>,
    dot_general=<function dot_general at 0x74eaf64bfce0>,
    dtype=None,
    in_features=64,
    kernel=Param(
      value=Array(shape=(5, 64, 32), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x74ea44264c20>,
    out_features=32,
    par

## State saving

In [25]:
from algorithms.utils import save_state_multiagent
save_state_multiagent(network_batteries, network_rec, config, params, val_info=out.get('val_info'), env_type=env_type, additional_info='')

In [26]:
out

{'metrics': {'actions': {'REC_agent': Array([[[[2.65489876e-01, 4.78627495e-02, 1.74074337e-01,
             4.28431928e-01, 8.41410905e-02],
            [7.66171575e-01, 8.88865616e-04, 3.44781642e-04,
             6.71613589e-02, 1.65433407e-01],
            [1.97537243e-01, 5.65253139e-01, 1.23179136e-02,
             1.77199081e-01, 4.76927124e-02],
            [4.26600836e-02, 1.41440630e-01, 2.10622158e-02,
             7.30913520e-01, 6.39236271e-02]],
   
           [[7.03076780e-01, 2.07242355e-01, 7.05523342e-02,
             1.03842672e-02, 8.74426775e-03],
            [3.60020667e-01, 2.82836050e-01, 1.58015832e-01,
             1.13767581e-02, 1.87750682e-01],
            [1.30484000e-01, 6.13429844e-01, 4.05008122e-02,
             1.53360385e-02, 2.00249344e-01],
            [5.72450221e-01, 2.51237750e-01, 9.37915146e-02,
             5.90006402e-03, 7.66204596e-02]],
   
           [[2.60162521e-02, 8.01015571e-02, 2.60967016e-01,
             1.75183922e-01, 4.5773124